**Part 3 - Mapping Closure in 2D**

This notebook implements the mapping closure developed by (Chen, H. 1989) and extended by (Pope, S.B. 1991) for a two-dimensional (2D) problem where the vector $\mathbf{Y} = (W,B)$ represents the random variables $W$ - the vertical velocity and $B$ - the buoyancy.

*Evolution equation*

The joint pdf $f_{\mathbf{Y}}$ evolves according to the forward Kolmogorov equation given by

\begin{equation}
\frac{\partial f_{\mathbf{Y}} }{ \partial t}= -\frac{\partial }{\partial y_{\alpha}} \mathbf{D}^{(1)}f_{\mathbf{Y}}  + \frac{\partial^{2}}{\partial y^{2}_{\alpha}} \mathbf{D}^{(2)}f_{\mathbf{Y}},
\end{equation}

where $\mathbf{D}^{(1)}$ arises due to diffusive fluxes at the boundary, which must be specified. In this case buoyancy forcing will produce a non-zero forcing, but there will be no forcing in $W$. The diffusion term $\mathbf{D}^{(2)}$ can be modelled using a mutlidimensional extension of the mapping closure following (Pope, S.B. 1991) as we now show. 

*Mapping closure*

Corresponding to the random vector $\mathbf{Y} = (Y_1,Y_2)$ we now have a vector of Gaussian random fields $\mathbf{Z} = (Z_1,Z_2)$. The (now multidimensional) mapping $\mathscr{Y}_{\alpha}(-,t):\mathbb{R}^{\alpha} \rightarrow \mathbb{R}$ which maps the Gaussian field(s) to the surrogate field $\tilde{Y}_{\alpha}$ is hierarchical in the sense that 
\begin{align*}
    \tilde{Y}_1(\mathbf{x},t) &= \mathscr{Y}_{1}(Z_1(J_1(t)\mathbf{x}),t), \\
    \tilde{Y}_2(\mathbf{x},t) &= \mathscr{Y}_{2}(Z_1(J_1(t)\mathbf{x}),Z_2(J_2(t)\mathbf{x}),t),
\end{align*}
such that $\mathscr{Y}_{\alpha}$ only depends on the subset $[\mathbf{Z}]_{\alpha}$

*Mapping equation*

The mapping $\mathscr{Y}_{\alpha}(-,t)$ evolves according to

\begin{equation}
\frac{\partial \mathscr{Y}_{\alpha} }{\partial t} = \mathbf{D}^{(1)}_{\alpha} + \sum_{j=1}^{\alpha} \frac{\gamma_{\alpha}}{\tau_{j}(t)} \left( \frac{\partial^{2}}{\partial z_{j}^{2}} - z_{j}\frac{\partial }{ \partial z_{j} }\right)\mathscr{Y}_{\alpha},
\end{equation}

which when expanded is given by


\begin{align*}
\frac{\partial \mathscr{Y}_1 }{\partial t} &= \mathbf{D}^{(1)}_{1} + \frac{\gamma_{1}}{\tau_1(t)} \left( \frac{\partial^{2}}{\partial z_{1}^{2}} - z_1\frac{\partial }{ \partial z_1 }\right)\mathscr{Y}_1, \\
\frac{\partial \mathscr{Y}_2 }{\partial t} &= \mathbf{D}^{(1)}_{2} + \gamma_2 \left[ \frac{1}{\tau_1(t)} \left( \frac{\partial^{2}}{\partial z_{1}^{2}} - z_1\frac{\partial }{ \partial z_1 }\right) + \frac{1}{\tau_2(t)} \left( \frac{\partial^{2}}{\partial z_{2}^{2}} - z_2\frac{\partial }{ \partial z_2 }\right) \right] \mathscr{Y}_2.
\end{align*}


**Setup**

We first import the required libraries.

In [17]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt

We then continue by defining a computational grid, on which the position vector $\mathbf{z} = (z_1,z_2)$ for the Gaussian random variables is defined.  

In [34]:
# Time scales (cf. Taylor microscale)
t1 = 1
t2 = 1

# Boundary conditions
bc1 = 'Neumann'
bc2 = 'Neumann'

# Numerical resolution
N1 = 32
N2 = 64

# Domain
z1 = np.linspace(-3, 3, N1)
z2 = np.linspace(-4, 4, N2)

# 2d grid for gaussian random variables Y1 and Y2
I2 = np.ones(N2)
I1 = np.ones(N1)
z1_2d = np.kron(z1,I2).reshape((N1,N2))
z2_2d = np.kron(I1,z2).reshape((N1,N2))

*Differential operators*

To define the differential operators we will use to build the previous equations right hand sides. This is done by defining the gradient and Laplacian:

In [35]:
# Define the gradient & Laplacian operators
def grad(x):
  """Gradient operator ∂/∂x"""
  N = len(x)
  dx= x[0] - x[1] 
  Akp1 =  np.ones(N-1)
  Akm1 = -np.ones(N-1)
  return sparse.diags( [Akm1,Akp1], [-1,1] )/2*dx

def laplacian(x, bcs = 'Dirichlet'):
  """Laplacian ∂^2/∂x^2"""
  
  N = len(x)
  dx= x[0] - x[1] 

  Akp1 = np.ones(N-1)
  Ak0  = -2*np.ones(N)
  Akm1 = np.ones(N-1)

  if bcs == 'Neumann':
      Akp1[0 ] = 2.0; # Use f_1     = f_-1    @ f_0
      Akm1[-1] = 2.0; # Use f_{N+1} = f_{N-1} @ f_N
  elif bcs == 'Dirichlet':
      Akp1[0]  = 0.
      Ak0[0]   = 0.; Ak0[-1] =0.
      Akm1[-1] = 0.

  return sparse.diags( [Akm1,Ak0,Akp1], [-1,0,1] )/dx**2

We then define the terms in the mapping equation by constructing the terms using Kronecker products as follows:

In [36]:
# Define the 1D equation operators 
D1_1d = grad(z1)
L1_1d = laplacian(z1, bcs = bc1)
Z1_1d = sparse.diags([z1],[0])
I1_1d = sparse.identity(len(z1))

D2_1d = grad(z2)
L2_1d = laplacian(z2, bcs = bc2)
Z2_1d = sparse.diags([z2],[0])
I2_1d = sparse.identity(len(z2))

# Take Kronecker products to obtain them in 2D
D1_2d = sparse.kron(D1_1d,I2_1d)
L1_2d = sparse.kron(L1_1d,I2_1d)
Z1_2d = sparse.kron(Z1_1d,I2_1d)

D2_2d = sparse.kron(I1_1d,D2_1d)
L2_2d = sparse.kron(I1_1d,L2_1d)
Z2_2d = sparse.kron(I1_1d,Z2_1d)

Using the predefined terms we then construct the right hand side operators for $\mathscr{Y}_1(z_1,t)$ and $\mathscr{Y}_2(z_1,z_2,t)$. For now we ignore the coefficients $\mathbf{D}^{(1)}_{\alpha}$ and concentrate on the differential operators only.

In [37]:
# Define the RHS operator for Y_1
L1 = (1/t1)*(L1_1d - Z1_1d@D1_1d)

# Define the RHS operator for Y_2
L2 = (1/t1)*(L1_2d - Z1_2d@D1_2d) + (1/t2)*(L2_2d - Z2_2d@D2_2d)

*Initial conditions & forcing*

Using $\mathbf{\eta}$ as a dummy variable, this mapping is chosen such that the conditional cumulative distribution function satisfies
\begin{equation}
  F_{\alpha|\alpha-1}(\mathscr{Y}_{\alpha}([\mathbf{\eta}]_{\alpha},t), t)=G(\eta_{\alpha}),
\end{equation}
which taking a derivative with respect to $\eta_{\alpha}$ implies that
\begin{equation}
    f_{\alpha|\alpha-1}([\mathbf{y}]_{\alpha},t)\frac{\partial \mathscr{Y}_{\alpha}}{\partial \eta_{\alpha}}\bigg|_{\mathbf{\eta}}=g(\eta_{\alpha})>0,
\end{equation}
and inturn that if the conditional density is strictly positive then $\partial_{\eta_{\alpha}}\mathscr{Y}_{\alpha}>0$.

To prescribe initial conditions for the mapping we make use of these relations by choosing
\begin{align*}
F_1(\tilde{y}_1) &= \frac{1}{2} [ 1 + \text{erf}(\frac{y_1}{\sigma_1 \sqrt{2}})  ], \\
F_{2|1}(\tilde{y}_2) &= \frac{1}{2} [ 1 + \text{erf}(\frac{y_2}{\sigma_2 \sqrt{2}})  ], 
\end{align*}
as these have a well defined inverse also known as the quantile function of the normal distribution and thus make it easy to invert for $\mathscr{Y}_1, \mathscr{Y}_2$. Implementing these relations below we calculate the initial conditions.

In [ ]:
from scipy.special import erf, erfinv

def G(z):
    return (1 + erf(z/np.sqrt(2)))/2

def g(z):
    return np.exp(-z**2/2)/np.sqrt(2*np.pi)

# Standard deviations for initial condition
σ1 = 4
Y1 = (erfinv(2*G(z1) - 1)*σ1*np.sqrt(2))

σ2 = 2
Y2 = (erfinv(2*G(z2_2d) - 1)*σ2*np.sqrt(2)).flatten()

# plt.plot(z1,Y1)
# plt.show()

# plt.pcolormesh(z1,z2,Y2.reshape((N1,N2)).T)
# plt.show()

We then pass these initial conditions to a time-stepping routine 

In [ ]:
dt = 0.001
nt = 1000

def one_to_2d(Y1):
    return np.kron(Y1,I2) 

for i in range(nt):
    Y2 += (L2 @ Y2 + one_to_2d(Y1))* dt
    Y1 +=  L1 @ Y1 * dt
    
    if i % 100 == 0:
        print('time t = ',i*dt)
        plt.plot(z1,Y1)
        plt.show()

        plt.pcolormesh(z1,z2,Y2.reshape((N1,N2)).T)
        plt.show()

After computing the time-evolution of the mapping we can now recover the joint distribution $f_Y(y1, y2)$.

In [ ]:
f = g(z1_2d).flatten() * g(z2_2d).flatten() * (D1_2d @ Y1_0_2d)**(-1) * (D2_2d @ Y2_0_2d)**(-1)

plt.contourf(unflatten(Y1_0_2d), unflatten(Y2_0_2d), unflatten(f))
plt.colorbar()
plt.show()
